In [1]:
import os, sys
from os.path import abspath

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

import tempfile
# Create a temporary directory for the model checkpoint. Remove the exising one if this cell is rerun
try:
    temp_model_dir.cleanup()
except NameError:
    print("Temporary directory not created yet")
finally:
    temp_model_dir = tempfile.TemporaryDirectory() 
    print("Temporary directory:", temp_model_dir.name)

Temporary directory not created yet
Temporary directory: /tmp/tmp2b9x4pw8


## Load Dataset and Model
We will load the CIFAR10 dataset and a pre-trained alexnet model.

In [2]:
from art.utils import load_dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('cifar10')

In [3]:
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import layers, models, losses

model = models.Sequential()

# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model.add(layers.Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=x_train.shape[1:]))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10))

model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
    

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

2022-02-07 16:43:28.342342: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:43:28.351111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:43:28.353603: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
from art.estimators.classification import KerasClassifier

classifier = KerasClassifier(clip_values=(min_, max_), model=model, use_logits=True)

In [5]:
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)
model.save(temp_model_dir.name + '/keras_model')

Train on 50000 samples


2022-02-07 16:43:30.348129: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-07 16:43:30.349775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:43:30.350843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:43:30.351734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/10


2022-02-07 16:43:31.934725: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-02-07 16:43:32.289362: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


50000/50000 [==============================] - 6s 112us/sample - loss: 1.6419 - accuracy: 0.3974
Epoch 2/10
50000/50000 [==============================] - 4s 81us/sample - loss: 1.2536 - accuracy: 0.5533
Epoch 3/10
50000/50000 [==============================] - 4s 80us/sample - loss: 1.0790 - accuracy: 0.6155
Epoch 4/10
50000/50000 [==============================] - 4s 80us/sample - loss: 0.9716 - accuracy: 0.6574
Epoch 5/10
50000/50000 [==============================] - 4s 80us/sample - loss: 0.8928 - accuracy: 0.6849
Epoch 6/10
50000/50000 [==============================] - 4s 81us/sample - loss: 0.8248 - accuracy: 0.7120
Epoch 7/10
50000/50000 [==============================] - 4s 80us/sample - loss: 0.7733 - accuracy: 0.7270
Epoch 8/10
50000/50000 [==============================] - 4s 80us/sample - loss: 0.7301 - accuracy: 0.7435
Epoch 9/10
50000/50000 [==============================] - 4s 81us/sample - loss: 0.6975 - accuracy: 0.7549
Epoch 10/10
50000/50000 [======================

2022-02-07 16:44:13.303925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:44:13.304327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:44:13.304579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:44:13.304894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 16:44:13.305151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

INFO:tensorflow:Assets written to: /tmp/tmp2b9x4pw8/keras_model/assets


In [6]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 76.83%


## Generate the Poison
We now will generate the poison using the hidden trigger backdoor attack. First, we define the target and source classes as well as the backdoor trigger. The target class will be the class we want to insert poisoned data into. The source class will be the class we will add a trigger to in order to cause misclassification into the target.

The backdoor trigger will be a small image patch inserted into the source class images. At test time, we should be able to use this trigger to cause the classifier to misclassify source class images with the trigger added as the target class.

In [7]:
from art.attacks.poisoning.backdoor_attack import PoisoningAttackBackdoor
target = np.array([0,0,0,0,1,0,0,0,0,0])
source = np.array([0,0,0,1,0,0,0,0,0,0])

# Backdoor Trigger Parameters
patch_size = 8
x_shift = 32 - patch_size - 5
y_shift = 32 - patch_size - 5

# Define the backdoor poisoning object. Calling backdoor.poison(x) will insert the trigger into x.
from art.attacks.poisoning import perturbations
def mod(x):
    original_dtype = x.dtype
    x = perturbations.insert_image(x, backdoor_path="../utils/data/backdoors/htbd.png",
                                   channels_first=False, random=False, x_shift=x_shift, y_shift=y_shift,
                                   size=(patch_size,patch_size), mode='RGB', blend=1)
    return x.astype(original_dtype)
backdoor = PoisoningAttackBackdoor(mod)

Here we run the attack. `eps` controls how much the target images can be perturbed with respect to an l-infinity distance. `feature_layer` dicates with layer's output will be used to define the attack's loss. It can either be the name of the layer or the layer index according to the ART estimator. `poison_percent` controls how many poisoned samples will be generated based on the size of the input data.

The attack will return poisoned inputs of the target class and the indicies in the data that those poisoned inputs should replace.

In [8]:
from art.attacks.poisoning import HiddenTriggerBackdoor
poison_attack = HiddenTriggerBackdoor(classifier, eps=16/255, target=target, source=source, feature_layer=9, backdoor=backdoor, learning_rate=0.01, decay_coeff = .1, decay_iter = 1000, max_iter=3000, batch_size=25, poison_percent=.015)

poison_data, poison_indices = poison_attack.poison(x_train, y_train)
print("Number of poison samples generated:", len(poison_data))

Hidden Trigger:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch:  0 | batch: 0 | i:     0 | LR: 0.01000 | Loss Val: 2687.496 | Loss Avg: 2687.496
Epoch:  0 | batch: 0 | i:   100 | LR: 0.01000 | Loss Val: 270.641 | Loss Avg: 374.225
Epoch:  0 | batch: 0 | i:   200 | LR: 0.01000 | Loss Val: 262.501 | Loss Avg: 319.566
Epoch:  0 | batch: 0 | i:   300 | LR: 0.01000 | Loss Val: 262.036 | Loss Avg: 299.013
Epoch:  0 | batch: 0 | i:   400 | LR: 0.01000 | Loss Val: 249.067 | Loss Avg: 287.744
Epoch:  0 | batch: 0 | i:   500 | LR: 0.01000 | Loss Val: 248.126 | Loss Avg: 281.043
Epoch:  0 | batch: 0 | i:   600 | LR: 0.01000 | Loss Val: 245.954 | Loss Avg: 276.038
Epoch:  0 | batch: 0 | i:   700 | LR: 0.01000 | Loss Val: 248.542 | Loss Avg: 271.945
Epoch:  0 | batch: 0 | i:   800 | LR: 0.01000 | Loss Val: 240.874 | Loss Avg: 268.217
Epoch:  0 | batch: 0 | i:   900 | LR: 0.01000 | Loss Val: 239.004 | Loss Avg: 265.211
Epoch:  0 | batch: 0 | i:  1000 | LR: 0.00100 | Loss Val: 240.615 | Loss Avg: 262.947
Epoch:  0 | batch: 0 | i:  1100 | LR: 0.00100 | Loss

## Finetune the Model
Now, we must finetune the model using the poisoned data and a small number of clean training inputs.  Here, we randomly select an equal number of training inputs from each of the classes.

In [9]:
# Create finetuning dataset
dataset_size = 2500
num_classes = 10
num_per_class = dataset_size/num_classes

poison_dataset_inds = []

for i in range(num_classes):
    class_inds = np.where(np.argmax(y_train,axis=1) == i)[0]
    num_select = int(num_per_class)
    if np.argmax(target) == i:
        num_select = int(num_select - min(num_per_class,len(poison_data)))
        poison_dataset_inds.append(poison_indices)
        
    if num_select != 0:
        poison_dataset_inds.append(np.random.choice(class_inds, num_select, replace=False))
    
poison_dataset_inds = np.concatenate(poison_dataset_inds)

poison_x = np.copy(x_train)
poison_x[poison_indices] = poison_data
poison_x = poison_x[poison_dataset_inds]

poison_y = np.copy(y_train)[poison_dataset_inds]

In [10]:
model = tf.keras.models.load_model(temp_model_dir.name + '/keras_model')
temp_model_dir.cleanup() # Remove the temporary directory after loading the checkpoint

model.trainable = False
model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [11]:
classifier = KerasClassifier(clip_values=(min_, max_), model=model)
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 76.83%


In [12]:
finetune_model = tf.keras.layers.Dense(10)(model.layers[-2].output)
finetune_model = tf.keras.Model(inputs=model.inputs, outputs=finetune_model)
finetune_model.summary()

lr = 0.5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

finetune_model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=["accuracy"])
finetune_classifier = KerasClassifier(clip_values=(min_, max_), model=finetune_model, use_logits=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928 

In [13]:
trigger_test_inds = np.where(np.all(y_test == source, axis=1))[0]

test_poisoned_samples, test_poisoned_labels  = backdoor.poison(x_test[trigger_test_inds], y_test[trigger_test_inds])


for i in range(4):
    predictions = finetune_classifier.predict(x_test)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on benign test examples: {}%".format(accuracy * 100))
    
    predictions = finetune_classifier.predict(x_test[trigger_test_inds])
    b_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds], axis=1)) / len(trigger_test_inds)
    print("Accuracy on benign trigger test examples: {}%".format(b_accuracy * 100))
    
    predictions = finetune_classifier.predict(test_poisoned_samples)
    p_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(test_poisoned_labels,axis=1)) / len(test_poisoned_labels)
    print("Accuracy on poison trigger test examples: {}%".format(p_accuracy * 100))
    p_success = np.sum(np.argmax(predictions, axis=1) == np.argmax(target)) / len(test_poisoned_labels)
    print("Success on poison trigger test examples: {}%".format(p_success * 100))
    print()
    print("Training Epoch", i)
    if i != 0:
        lr *= 0.1
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        finetune_model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=["accuracy"])
    finetune_classifier = KerasClassifier(clip_values=(min_, max_), model=finetune_model, use_logits=True)

    finetune_classifier.fit(poison_x, poison_y, nb_epochs=1)


Accuracy on benign test examples: 5.19%
Accuracy on benign trigger test examples: 2.8000000000000003%
Accuracy on poison trigger test examples: 4.0%
Success on poison trigger test examples: 12.2%

Training Epoch 0
Train on 2500 samples
2500/2500 [==============================] - 0s 99us/sample - loss: 7.3416 - accuracy: 0.6140
Accuracy on benign test examples: 70.53%
Accuracy on benign trigger test examples: 62.9%
Accuracy on poison trigger test examples: 25.2%
Success on poison trigger test examples: 33.4%

Training Epoch 1
Train on 2500 samples
2500/2500 [==============================] - 0s 94us/sample - loss: 6.8855 - accuracy: 0.7032
Accuracy on benign test examples: 74.77000000000001%
Accuracy on benign trigger test examples: 48.199999999999996%
Accuracy on poison trigger test examples: 15.299999999999999%
Success on poison trigger test examples: 45.300000000000004%

Training Epoch 2
Train on 2500 samples
2500/2500 [==============================] - 0s 95us/sample - loss: 5.9205

In [14]:
print("Final Performance")
predictions = finetune_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

predictions = finetune_classifier.predict(x_test[trigger_test_inds])
b_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds], axis=1)) / len(trigger_test_inds)
print("Accuracy on benign trigger test examples: {}%".format(b_accuracy * 100))

predictions = finetune_classifier.predict(test_poisoned_samples)
p_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(test_poisoned_labels,axis=1)) / len(test_poisoned_labels)
print("Accuracy on poison trigger test examples: {}%".format(p_accuracy * 100))
p_success = np.sum(np.argmax(predictions, axis=1) == np.argmax(target)) / len(test_poisoned_labels)
print("Success on poison trigger test examples: {}%".format(p_success * 100))

Final Performance
Accuracy on benign test examples: 74.85000000000001%
Accuracy on benign trigger test examples: 49.7%
Accuracy on poison trigger test examples: 16.0%
Success on poison trigger test examples: 44.0%
